In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader, Dataset 
from torchvision import datasets, transforms

# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 5
kernel_size = 3
stride = 2

## 모듈화  
코드의 가독성을 좋게 하고 보수 및 관리를 쉽게 합니다.  

```
+-- configs/
|   +-- cnn.yaml
+-- dataset
|   +-- mnist_data/...
|   +-- MNIST_LOADER.py
+-- models
|   +-- cnn.py
|   +-- res_net.py
+-- runs
|   +-- cnn/...
|   +-- experiments/...
+-- wetights
|   +-- cnn.pth
+-- argparse_tutorial.py
+-- README.md
+-- run_cnn.py
+-- run_cnn2.py
+-- requirements.txt
+-- utils.py
```

## Quiz (Easy)  
위 구조와 같이 모듈화를 진행하겠습니다.  
1) dataset 폴더를 만들고 mnist_data 폴더를 옮겨주세요. 그리고 MNIST_LOADER.py를 dataset에 만듭니다.   
2) models 폴더를 만들고 cnn.py, res_net.py를 만들고 내용을 채워주세요   
3) utils 파일을 만들고 conv2d_size_out 함수를 작성하세요. CNN 클래스의 생성자에 정의된 함수를 그대로 옮기면 됩니다.  # 지금 안합니다.
4) runs 폴더를 만들어주세요.   
5) weights 폴더를 만들어주세요.   
6) requirements.txt 파일을 만들어주세요.    

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, download=True, transform=transforms.ToTensor())
valid_dataset = datasets.MNIST(root='./mnist_data/', train=False, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
vaild_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [11]:
def train(epoch, model, loss_func, train_loader, optimizer):
    model.train()
    for batch_index, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()
        if batch_index % 100 == 0:
            print(f'Train Epoch: {epoch+1} | Batch Status: {batch_index*len(x)}/{len(train_loader.dataset)} \
            ({100. * batch_index * batch_size / len(train_loader.dataset):.0f}% | Loss: {loss.item():.6f}')

def test(model, loss_func, test_loader):
    model.eval()
    test_loss = 0
    correct_count = 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        test_loss += loss_func(y_pred, y).item()
        pred = y_pred.data.max(1, keepdim=True)[1]
        # torch.eq : Computes element-wise equality. return counts value
        correct_count += pred.eq(y.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print(f'=======================\n Test set: Average loss: {test_loss:.4f}, Accuracy: {correct_count/len(test_loader.dataset):.3}')

In [ ]:
# class CNN(nn.Module):
#     def __init__(self, C, W, H, K, S): # 채널, 너비, 높이, 커널 사이즈, 스트라이드
#         super(CNN, self).__init__()
#         # nn.Module에는 이미 conv 레이어가 구현되어 있다. 
#         # 배치정규화도 구현되어있고 다 구현되어있습니다. 
#         self.conv1 = nn.Conv2d(C, 32, kernel_size=K, stride=S)
#         self.bn1 = nn.BatchNorm2d(32)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=K, stride=S)
#         self.bn2 = nn.BatchNorm2d(64)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=K, stride=S)
#         self.bn3 = nn.BatchNorm2d(128)
        
#         def conv2d_size_out(size, kernel_size=K, stride=S):
#             print((size - (kernel_size - 1) - 1) // stride + 1)
#             return (size - (kernel_size - 1) - 1) // stride + 1
        
#         convw = conv2d_size_out(W, K, S)
#         convw = conv2d_size_out(convw, K, S)
#         convw = conv2d_size_out(convw, K, S)
        
#         self.linear_input_size = convw * convw * 128
#         self.fc = nn.Linear(self.linear_input_size, 10)
        
#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = x.view(x.size(0), -1) # (batch_size, flatten_size)
#         x = F.relu(self.fc(x))
#         return F.log_softmax(x)

In [12]:
%%writefile run_cnn3.py


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


from models.cnn import CNN
from dataset.MNIST_LOADER import make_loader
import argparse
import yaml

def train(epoch, model, loss_func, train_loader, optimizer):
    model.train()
    for batch_index, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()
        if batch_index % 100 == 0:
            print(f'Train Epoch: {epoch+1} | Batch Status: {batch_index*len(x)}/{len(train_loader.dataset)} \
            ({100. * batch_index * batch_size / len(train_loader.dataset):.0f}% | Loss: {loss.item():.6f}')

def test(model, loss_func, test_loader):
    model.eval()
    test_loss = 0
    correct_count = 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        test_loss += loss_func(y_pred, y).item()
        pred = y_pred.data.max(1, keepdim=True)[1]
        # torch.eq : Computes element-wise equality. return counts value
        correct_count += pred.eq(y.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print(f'=======================\n Test set: Average loss: {test_loss:.4f}, Accuracy: {correct_count/len(test_loader.dataset):.3}')

# parser 정의
parser = argparse.ArgumentParser(description='Argparse Tutorial')

parser.add_argument('--config_path', type=str, default='configs/cnn.yaml', help = "Score of english")

args = parser.parse_args()

train_loader, vaild_loader, test_loader, shape = make_loader(16)
C = shape[0]
W = shape[1]
H = shape[2]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device('cpu')

cnn = CNN(C=C, W=W, H=H, K=3, S=2) 
cnn = cnn.to(device)
ce_loss = nn.CrossEntropyLoss()

# with 구문으로 파일을 불러옵니다.

with open(args.config_path) as f:
    config = yaml.load(f,Loader=yaml.FullLoader)
    print(type(config))

# Hyperparameters
batch_size = config['batch_size']
learning_rate = config['learning_rate']
epochs = config['epochs']
kernel_size = config['kernel_size']
stride = config['stride']

optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)

for epoch in range(epochs):
    train(epoch, cnn, ce_loss, train_loader, optimizer)

test(cnn, ce_loss, test_loader)

Overwriting run_cnn3.py


In [13]:
!python3 run_cnn3.py

channel: 1, width: 28, height: 28
13
6
2
<class 'dict'>
/home/hchang/Working/Python3/Pytorch-Master/Practice/models/cnn.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Train Epoch: 1 | Batch Status: 0/60000             (0% | Loss: 2.345618
Train Epoch: 1 | Batch Status: 1600/60000             (3% | Loss: 0.768841
Train Epoch: 1 | Batch Status: 3200/60000             (5% | Loss: 0.523701
Train Epoch: 1 | Batch Status: 4800/60000             (8% | Loss: 1.301229
Train Epoch: 1 | Batch Status: 6400/60000             (11% | Loss: 1.063578
Train Epoch: 1 | Batch Status: 8000/60000             (13% | Loss: 0.363229
Train Epoch: 1 | Batch Status: 9600/60000             (16% | Loss: 1.022959
Train Epoch: 1 | Batch Status: 11200/60000             (19% | Loss: 0.339684
Train Epoch: 1 | Batch Status: 12800/60000             (21% | Loss: 0.050853
Train Epoch: 1 | Batch Status: 14400/60000 